In [1]:
## Load credentials from config/credentials.ini
#redo3.py
#Author: Graham Waters, 2023
#^ Essential Imports
import asyncio
import configparser
import logging
import os
import random
import sys
import time
import traceback
from datetime import datetime
#^ Non-Essential Imports
import alive_progress
import pandas as pd
import pandas_ta
import pandas_ta as ta
from alive_progress import alive_bar
from icecream import ic
from pytz import timezone
from robin_stocks import robinhood as r
from tqdm import tqdm
import numpy as np
#^ Constants
PCT_SPEND = 0.05 # 5% of buying power
verbose_mode = True # Set to True to see more logging output
from colorama import Back, Fore, Style
from ratelimit import limits, sleep_and_retry
config = configparser.ConfigParser()
config.read('config/credentials.ini')
# Define variables from config
# Define variables from config
coins = [coin.strip() for coin in config['trading']['coins'].split(', ')]
stop_loss_percent = float(config['trading']['stop_loss_percent'])
percent_to_use = float(config['trading']['percent_to_use'])
verbose_mode = config['logging'].getboolean('verbose_mode')
debug_verbose = config['logging'].getboolean('debug_verbose')
reset_positions = config['logging'].getboolean('reset_positions')
minimum_usd_per_position = float(config['trading']['minimum_usd_per_position'])
pct_to_buy_with = float(config['trading']['percent_to_use'])
pct_to_buy_per_trade = float(config['trading']['percent_to_spend_per_trade'])
username = config['robinhood']['username']
password = config['robinhood']['password']
login = r.login(username, password)
print(f"Logged in as {username}")
# Function to get current price for a coin


Logged in as "graham.waters37@gmail.com"


# MicroStrategy Two: The OverWatchman
This looping bot will watch the price of the coin and will adopt these roles:
- The Harvester: if it sees a price that has a Z-Score greater than 2.0 and a price that is greater than the 20 day moving average, it will sell the coin's position (100%) and terminate any outstanding orders.
- The Planter: If the bot sees a price that has a Z-Score less than -2.0 and a price that is less than the 20 day moving average, it will buy the minimum_usd_per_trade (if it can afford it) as this is a dip. The planter stage is more intensive than the harvester as it must track the price following the purchase, and if it continues to go down, it keeps buying more with decreasing investment volumes (in USD) by a factor of 10 percent per purchase. It will continue to buy until the price goes up or it runs out of money.

In [3]:
import schedule
import time
from robin_stocks import robinhood as r
import pandas as pd
import configparser
from datetime import datetime

# Load credentials from config/credentials.ini
config = configparser.ConfigParser()
config.read('config/credentials.ini')

# Define variables from config
coins = [coin.strip() for coin in config['trading']['coins'].split(', ')]
minimum_usd_per_trade = float(config['trading']['minimum_usd_per_trade'])

# Function to get current price for a coin
@sleep_and_retry
def get_current_price(coin_name):
    quote = r.crypto.get_crypto_quote(coin_name)
    return float(quote['mark_price'])

# Function to buy a coin
@sleep_and_retry
def buy_coin(coin_name, amount):
    current_price = get_current_price(coin_name)
    amount_to_buy = float(amount) / float(current_price)
    buying_usd = round(float(amount_to_buy), 2)
    buy_cost = round(float(buying_usd) * float(current_price), 2)
    result = r.orders.order_crypto(
        symbol=str(coin_name).upper(),
        amountIn='dollars',
        side='buy',
        quantityOrPrice=float(buy_cost),
        limitPrice=float(current_price),
        timeInForce='gtc',
        jsonify=True
    )
    return result

# Function to sell a coin
@sleep_and_retry
def sell_coin(coin_name):
    current_price = get_current_price(coin_name)
    holdings = r.crypto.get_crypto_positions(info=None)
    coin_holdings = [coin for coin in holdings if coin['currency']['code'] == coin_name]
    amount_to_sell = float(coin_holdings[0]['quantity'])
    result = r.orders.order_crypto(
        symbol=coin_name,
        amountIn='quantity',
        side='sell',
        quantityOrPrice=float(amount_to_sell),
        limitPrice=float(current_price),
        timeInForce='gtc',
        jsonify=True
    )
    return result

def strategy():
    global minimum_usd_per_trade  # Add this line

    # Load credentials from config/credentials.ini
    config = configparser.ConfigParser()
    config.read('config/credentials.ini')

    # Define variables from config
    coins = [coin.strip() for coin in config['trading']['coins'].split(', ')]
    minimum_usd_per_trade = float(config['trading']['minimum_usd_per_trade'])
    # this strategy can use day
    for coin in tqdm(coins):
        # Get the historical data for the coin
        historicals = r.crypto.get_crypto_historicals(coin, interval='5minute', span='week', bounds='24_7', info=None)
        df = pd.DataFrame(historicals)
        df['close_price'] = df['close_price'].astype(float)
        
        # Calculate the 20 day moving average
        df['moving_avg'] = df['close_price'].rolling(window=20).mean()
        
        # Calculate the Z-Score
        df['z_score'] = (df['close_price'] - df['moving_avg']) / df['close_price'].std(ddof=0)
        
        # Get the current price and Z-Score
        current_price = get_current_price(coin)
        current_z_score = df['z_score'].iloc[-1]
        
        # The Harvester
        if current_z_score > 2.0 and current_price > df['moving_avg'].iloc[-1]:
            # Sell the coin's position (100%) and terminate any outstanding orders
            sell_coin(coin_name=coin)
            print(Fore.YELLOW + f'Harvester Strategy: Selling {coin} at {current_price}' + Fore.RESET)
            print(Fore.YELLOW + f'Harvester Strategy: Terminating any outstanding orders for {coin}' + Fore.RESET)
            # if the time is not 00:00 to 1:00 Cancel all orders
            if datetime.now().hour != 0:
                r.orders.cancel_all_crypto_orders(coin)
        
        # The Planter
        if current_z_score < -2.0 and current_price < df['moving_avg'].iloc[-1]:
            # Buy the minimum_usd_per_trade (if it can afford it) as this is a dip
            buy_coin(coin,  float(config['trading']['minimum_usd_per_trade']))
            
            # Track the price following the purchase, and if it continues to go down, keep buying more
            while current_price < df['moving_avg'].iloc[-1]:
                current_price = get_current_price(coin)
                buy_coin(coin, minimum_usd_per_trade)
                minimum_usd_per_trade *= 0.9  # Decrease investment volume by 10 percent per purchase
                # daytime is defined as 9:30 AM ET to 7 PM ET
                interval = int(config['trading']['daytime_interval']) * 60 if datetime.now().hour >= 9 and datetime.now().hour <= 19 else int(config['trading']['nighttime_interval']) * 60
                # cast interval to int
                interval = int(interval) #note: just in case
                for i in tqdm(range(int(config['trading']['interval']) * 60)):
                    time.sleep(1)

# Function to execute the strategy
def execute_strategy():
    # schedule this for every hour
    schedule.every().hour.do(strategy)
# Schedule the strategy
execute_strategy()

# Run the scheduled jobs in a loop
while True:
    schedule.run_pending()
    time.sleep(60)  # Wait for 60 seconds before the next iteration


100%|██████████| 14/14 [00:12<00:00,  1.13it/s]


KeyboardInterrupt: 